In [33]:
from pandas import read_csv
from os import path
from os import listdir, chdir
from collections import OrderedDict
from pandas import DataFrame
import matplotlib.pyplot as plt

In [39]:
task_events_csv_colnames = ['time', 'missing', 'job_id', 'task_idx', 'machine_id', 'event_type', 'user', 'sched_cls', 
                            'priority', 'cpu_requested', 'mem_requested', 'disk', 'restriction']         
task_event_df = read_csv(path.join('task_events', 'part-00000-of-00500.csv.gz'), header=None, index_col=False, 
                         compression='gzip', names=task_events_csv_colnames)

maxtime_stamp = max(task_event_df['time'])
print maxtime_stamp

5611086346


In [4]:
from random import randint, sample, seed
seed(83)
# sample_moments = range(0,maxtime_stamp,1000000) # lay mau theo giay
sample_moments = sorted(sample(xrange(maxtime_stamp), 200))
snapshot_moment = randint(0, maxtime_stamp)  # lay mau ngau nhien 1 thoi diem
print snapshot_moment

287272106


In [5]:
# doc du lieu tu task_event, tinh tong tat ca cac CPU va RAM request cho tat cac cac job tai tung thoi diem
#
tasks_dict = {}
samples_dicts = OrderedDict([])
sample_moments_iterator = iter(sample_moments)
current_sample_moment = next(sample_moments_iterator)
tasks_df = None # day la fram ung voi lai 1 thoi diem lay mau ngau nhien (snapshot_moment)
# Not the most elegant code I've ever written...
# %%time
for fn in sorted(listdir('task_events')):
    
    fp = path.join('task_events', fn)
    task_events_df = read_csv(fp, header=None, index_col=False, compression='gzip', 
                              names=task_events_csv_colnames)
    
    for index, event in task_events_df.iterrows():
        
        if current_sample_moment is not None and event['time'] > current_sample_moment:
            tmp_tasks_df = DataFrame(tasks_dict.values())
            samples_dicts[current_sample_moment] = ({'time' : current_sample_moment, 
                                                     'cpu_requested' : (tmp_tasks_df['cpu_requested']).sum(), 
                                                     'mem_requested' : (tmp_tasks_df['mem_requested']).sum()})
            try:
                current_sample_moment = next(sample_moments_iterator)
            except StopIteration:
                current_sample_moment = None
                
        if tasks_df is None and event['time'] > snapshot_moment:
            tasks_df = DataFrame(tasks_dict.values())
            
        if event['event_type'] in [0, 7, 8]:
            tasks_dict[(event['job_id'], event['task_idx'])] = {'task_id' : (event['job_id'], event['task_idx']),
                                                                'machine_id' : event['machine_id'],
                                                                'cpu_requested' : event['cpu_requested'], 
                                                                'mem_requested' : event['mem_requested']}
        elif event['event_type'] in [2, 3, 4, 5, 6]:
            del tasks_dict[(event['job_id'], event['task_idx'])]
            
    if tasks_df is not None and current_sample_moment is None:
        break
                    
samples_df = DataFrame(samples_dicts.values())

print tasks_df.info()
print samples_df.info()

KeyboardInterrupt: 

In [10]:
# %%time
fig = plt.figure()
ax = fig.add_subplot(111)
ax.plot(samples_df['time'], samples_df['cpu_requested'], label='cpu requested')
ax.plot(samples_df['time'], samples_df['mem_requested'], label='mem requested')
plt.xlim(min(samples_df['time']), max(samples_df['time']))
plt.legend()
plt.show()

In [47]:
machines_dict = {}
sample_moments_iterator = iter(sample_moments)
current_sample_moment = next(sample_moments_iterator)
machines_df = None

machine_events_csv_colnames=['time', 'machine_id', 'event_type', 'platform_id', 'cpu', 'mem']

# %%time
for fn in sorted(listdir('machine_events')):
    fp = path.join('machine_events', fn)
    machine_events_df = read_csv(fp, header=None, index_col=False, compression='gzip', names=machine_events_csv_colnames)
    for index, event in machine_events_df.iterrows():
        
        if current_sample_moment is not None and event['time'] > current_sample_moment:
            tmp_machines_df = DataFrame(machines_dict.values())
            samples_dicts[current_sample_moment].update({'cpu_available' : (tmp_machines_df['cpu']).sum(), 
                                                         'mem_available' : (tmp_machines_df['mem']).sum()})
            try:
                current_sample_moment = next(sample_moments_iterator)
            except StopIteration:
                current_sample_moment = None
                
        if machines_df is None and event['time'] > snapshot_moment: 
            machines_df = DataFrame(machines_dict.values())
            
        if event['event_type'] in [0, 2]:
            machines_dict[event['machine_id']] = {'machine_id' : event['machine_id'], 
                                                  'cpu' : event['cpu'], 'mem' : event['mem']}
        elif event['event_type'] in [1]:
            del machines_dict[event['machine_id']]

    if machines_df is not None and current_sample_moment is None:
        break
                    
machines_df = DataFrame(machines_dict.values())
samples_df = DataFrame(samples_dicts.values())

print machines_df.info()
print
print samples_df.info()
print

In [9]:
fig = plt.figure()
ax = fig.add_subplot(111)
ax.plot(samples_df['time'], samples_df['cpu_requested'], label='cpu requested')
ax.plot(samples_df['time'], samples_df['cpu_available'], label='cpu available')
ax.plot(samples_df['time'], samples_df['mem_requested'], label='mem requested')
ax.plot(samples_df['time'], samples_df['mem_available'], label='mem available')
plt.xlim(min(samples_df['time']), max(samples_df['time']))
plt.legend()
plt.show()

KeyError: u'no item named cpu_available'

In [66]:
task_usage_csv_colnames=['starttime', 'endtime', 'job_id', 'task_idx', 'machine_id', 'cpu_usage', 'mem_usage', 
                         'assigned_mem', 'unmapped_cache_usage', 'page_cache_usage', 'max_mem_usage', 'disk_io_time', 
                         'max_disk_space', 'max_cpu_usage', 'max_disk_io_time', 'cpi', 'mai', 'sampling_rate', 'agg_type']

for moment in samples_dicts:
    samples_dicts[moment].update({'cpu_usage' : 0.0, 'mem_usage' : 0.0})
    
for task in tasks_dict:
    tasks_dict[task].update({'cpu_usage' : 0.0, 'mem_usage' : 0.0, 'in_events' : True, 'in_usage' : False})
    
# %%time
for fn in sorted(listdir('task_usage')):

    fp = path.join('task_usage', fn)
    task_usage_df = read_csv(fp, header=None, index_col=False, compression='gzip', names=task_usage_csv_colnames)

    laststart = max(task_usage_df['starttime'])
    firststart = min(task_usage_df['starttime'])
    # if laststart > max(sample_moments) and laststart > snapshot_moment:
    #     break
    if firststart > max(sample_moments) and firststart > snapshot_moment:
        break
    for moment in samples_dicts:
        task_usage_moment_df = task_usage_df[(task_usage_df['starttime'] <= moment) & 
                                             (moment <= task_usage_df['endtime'])]
        # print task_usage_moment_df.info()
        samples_dicts[moment]['cpu_usage'] += sum(task_usage_moment_df['cpu_usage'])
        samples_dicts[moment]['mem_usage'] += sum(task_usage_moment_df['mem_usage'])

    task_usage_snapshot_df = task_usage_df[(task_usage_df['starttime'] <= snapshot_moment) &
                                           (snapshot_moment <= task_usage_df['endtime'])]
    for index, usage in task_usage_snapshot_df.iterrows():
        task_id = (usage['job_id'], usage['task_idx'])
        if task_id in tasks_dict:
            tasks_dict[task_id].update({'cpu_usage' : usage['cpu_usage'], 'mem_usage' : usage['mem_usage'], 
                                        'in_events': True, 'in_usage' : True})
        else:
            tasks_dict[task_id] = {'cpu_requested' : 0.0, 'mem_requested' : 0.0, 
                                   'cpu_usage' : usage['cpu_usage'], 'mem_usage' : usage['mem_usage'],
                                   'in_events' : False, 'in_usage' : True}


samples_df = DataFrame(samples_dicts.values())
print samples_df.info()
tasks_df = DataFrame(tasks_dict.values())
print tasks_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 200 entries, 0 to 199
Data columns (total 7 columns):
cpu_available    200 non-null float64
cpu_requested    200 non-null float64
cpu_usage        200 non-null float64
mem_available    200 non-null float64
mem_requested    200 non-null float64
mem_usage        200 non-null float64
time             200 non-null int64
dtypes: float64(6), int64(1)None
<class 'pandas.core.frame.DataFrame'>
Int64Index: 142830 entries, 0 to 142829
Data columns (total 8 columns):
cpu_requested    142543 non-null float64
cpu_usage        142830 non-null float64
in_events        142830 non-null bool
in_usage         142830 non-null bool
machine_id       86485 non-null float64
mem_requested    142543 non-null float64
mem_usage        142830 non-null float64
task_id          142830 non-null object
dtypes: bool(2), float64(5), object(1)

None


In [69]:
fig = plt.figure()
ax = fig.add_subplot(111)
ax.plot(samples_df['time'], samples_df['cpu_requested'], label='cpu requested')
ax.plot(samples_df['time'], samples_df['cpu_available'], label='cpu available')
ax.plot(samples_df['time'], samples_df['cpu_usage'], label='cpu usage')
plt.xlim(min(samples_df['time']), max(samples_df['time']))
plt.legend()
plt.show()

In [59]:
fig = plt.figure()
ax = fig.add_subplot(111)
ax.plot(samples_df['time'], samples_df['mem_requested'], label='mem requested')
ax.plot(samples_df['time'], samples_df['mem_available'], label='mem available')
ax.plot(samples_df['time'], samples_df['mem_usage'], label='mem usage')
plt.xlim(min(samples_df['time']), max(samples_df['time']))
plt.legend()
plt.show()

In [70]:
demand_pct = [100.0 * cpur / cpua for cpur, cpua in zip(samples_df['cpu_requested'], samples_df['cpu_available'])]
usage_pct = [100.0 * cpuu / cpua for cpuu, cpua in zip(samples_df['cpu_usage'], samples_df['cpu_available'])]
fig = plt.figure()
ax = fig.add_subplot(111)
ax.plot(samples_df['time'], demand_pct, label='cpu % demand')
ax.plot(samples_df['time'], usage_pct, label='cpu % usage')
plt.xlim(min(samples_df['time']), max(samples_df['time']))
plt.legend()
plt.show()

In [75]:
demand_pct = [100.0 * memr / mema for memr, mema in zip(samples_df['mem_requested'], samples_df['mem_available'])]
usage_pct = [100.0 * memu / mema for memu, mema in zip(samples_df['mem_usage'], samples_df['mem_available'])]
fig = plt.figure()
ax = fig.add_subplot(111)
ax.plot(samples_df['time'], demand_pct, label='mem % demand')
ax.plot(samples_df['time'], usage_pct, label='mem % usage')
plt.xlim(min(samples_df['time']), max(samples_df['time']))
plt.legend()
plt.show()

In [7]:
# doc du lieu tu task_event, chon mot Machine ID
# roi tinh tong tat ca cac CPU va RAM request cho tat cac cac job tai tung thoi diem
machine_id = 1268205
tasks_dict = {}
samples_dicts = OrderedDict([])
sample_moments_iterator = iter(sample_moments)
current_sample_moment = next(sample_moments_iterator)
tasks_df = None # day la fram ung voi lai 1 thoi diem lay mau ngau nhien (snapshot_moment)
# Not the most elegant code I've ever written...
# %%time
task_events_csv_colnames = ['time', 'missing', 'job_id', 'task_idx', 'machine_id', 'event_type', 'user', 'sched_cls', 
                            'priority', 'cpu_requested', 'mem_requested', 'disk', 'restriction']  
for fn in sorted(listdir('task_events')):
    
    fp = path.join('task_events', fn)
    print 'reading file ' + fp
    task_events_df = read_csv(fp, header=None, index_col=False, compression='gzip', 
                              names=task_events_csv_colnames)
    
    for index, event in task_events_df.iterrows():
        
        if current_sample_moment is not None and event['time'] > current_sample_moment:
            tmp_tasks_df = DataFrame(tasks_dict.values())
            tmp_tasks_df = tmp_tasks_df[tmp_tasks_df['machine_id']==machine_id]
            # dong duoi nay co the toi uu hon
            samples_dicts[current_sample_moment] = ({'time' : current_sample_moment, 
                                                     'cpu_requested' : (tmp_tasks_df['cpu_requested']).sum(), 
                                                     'mem_requested' : (tmp_tasks_df['mem_requested']).sum(),
                                                     'disk_space_requested': (tmp_tasks_df['disk']).sum(),
                                                     'number_of_task_requested': len(tmp_tasks_df['machine_id'])})
            try:
                current_sample_moment = next(sample_moments_iterator)
            except StopIteration:
                current_sample_moment = None
                

        # if event['machine_id']== machine_id:
        if event['event_type'] in [0, 7, 8]:
            tasks_dict[(event['job_id'], event['task_idx'])] = {'task_id' : (event['job_id'], event['task_idx']),
                                                                    'machine_id' : event['machine_id'],
                                                                    'cpu_requested' : event['cpu_requested'], 
                                                                    'mem_requested' : event['mem_requested'],
                                                                    'disk': event['disk']}
        elif event['event_type'] in [2, 3, 4, 5, 6]:
            try:
                del tasks_dict[(event['job_id'], event['task_idx'])]
            except:
                    # print "loi:\n"
                    # print event
                pass
                    
    if tasks_df is not None and current_sample_moment is None:
        break
                    
samples_df = DataFrame(samples_dicts.values())
print samples_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 200 entries, 0 to 199
Data columns (total 5 columns):
cpu_requested               200 non-null float64
disk                        200 non-null float64
mem_requested               200 non-null float64
number_of_task_requested    200 non-null int64
time                        200 non-null int64
dtypes: float64(3), int64(2)None


In [22]:
# %%time
fig = plt.figure()
ax = fig.add_subplot(111)
ax.plot(samples_df['time'], samples_df['cpu_requested'], label='cpu requested')
ax.plot(samples_df['time'], samples_df['mem_requested'], label='mem requested')
ax.plot(samples_df['time'], samples_df['disk_space_requested'], label='disk space requested')
# ax.plot(samples_df['time'], samples_df['number_of_task_requested'], label='number of task')
plt.xlim(min(samples_df['time']), max(samples_df['time']))
plt.title('machine ID = ' +str(machine_id))
plt.legend()
plt.show()

In [28]:
#lay thong tin CPU, RAM available cua machine
machine_id = 1268205
machines_dict = {}
sample_moments_iterator = iter(sample_moments)
current_sample_moment = next(sample_moments_iterator)
machines_df = None

machine_events_csv_colnames=['time', 'machine_id', 'event_type', 'platform_id', 'cpu', 'mem']

# %%time
for fn in sorted(listdir('machine_events')):
    fp = path.join('machine_events', fn)
    machine_events_df = read_csv(fp, header=None, index_col=False, compression='gzip', names=machine_events_csv_colnames)
    for index, event in machine_events_df.iterrows():
        
        if current_sample_moment is not None and event['time'] > current_sample_moment:
            tmp_machines_df = DataFrame(machines_dict.values())
            if not tmp_machines_df.empty:
                samples_dicts[current_sample_moment].update({'cpu_available' : (tmp_machines_df['cpu']).sum(), 
                                                         'mem_available' : (tmp_machines_df['mem']).sum()})
            else:
                samples_dicts[current_sample_moment].update({'cpu_available' : 0, 
                                                         'mem_available' : 0})
            try:
                current_sample_moment = next(sample_moments_iterator)
            except StopIteration:
                current_sample_moment = None
        if event['machine_id'] == machine_id:
            if event['event_type'] in [0, 2]:
                machines_dict[event['machine_id']] = {'machine_id' : event['machine_id'], 
                                                      'cpu' : event['cpu'], 'mem' : event['mem']}
            elif event['event_type'] in [1]:
                del machines_dict[event['machine_id']]

    if machines_df is not None and current_sample_moment is None:
        break
                    
samples_df = DataFrame(samples_dicts.values())
print samples_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 200 entries, 0 to 199
Data columns (total 5 columns):
cpu_available    200 non-null float64
cpu_requested    200 non-null float64
mem_available    200 non-null float64
mem_requested    200 non-null float64
time             200 non-null int64
dtypes: float64(4), int64(1)None



In [31]:
fig = plt.figure()
ax = fig.add_subplot(111)
ax.plot(samples_df['time'], samples_df['cpu_requested'], label='cpu requested')
ax.plot(samples_df['time'], samples_df['cpu_available'], label='cpu available')
ax.plot(samples_df['time'], samples_df['mem_requested'], label='mem requested')
ax.plot(samples_df['time'], samples_df['mem_available'], label='mem available')
plt.xlim(min(samples_df['time']), max(samples_df['time']))
plt.legend()
plt.show()

In [35]:
# doc resource usage cho mot machine bat ky
machine_id = 1268205
task_usage_csv_colnames=['starttime', 'endtime', 'job_id', 'task_idx', 'machine_id', 'cpu_usage', 'mem_usage', 
                         'assigned_mem', 'unmapped_cache_usage', 'page_cache_usage', 'max_mem_usage', 'disk_io_time', 
                         'mean_local_disk_space', 'max_cpu_usage', 'max_disk_io_time', 'cpi', 'mai', 'sampling_portion', 
                         'agg_type', 'sampled_cpu_usage']

for moment in samples_dicts:
    samples_dicts[moment].update({'cpu_usage' : 0.0, 'mem_usage' : 0.0, 'disk_io_time': 0.0, 'disk_space': 0.0})
    
# %%time
for fn in sorted(listdir('task_usage')):

    fp = path.join('task_usage', fn)
    task_usage_df = read_csv(fp, header=None, index_col=False, compression='gzip', names=task_usage_csv_colnames)
    task_usage_df = task_usage_df[task_usage_df['machine_id']==machine_id]
    
    laststart = max(task_usage_df['starttime'])
    firststart = min(task_usage_df['starttime'])
    # if laststart > max(sample_moments) and laststart > snapshot_moment:
    #     break
    if firststart > max(sample_moments) and firststart > snapshot_moment:
        break
    for moment in samples_dicts:
        task_usage_moment_df = task_usage_df[(task_usage_df['starttime'] <= moment) & 
                                             (moment <= task_usage_df['endtime'])]
        # print task_usage_moment_df.info()
        samples_dicts[moment]['cpu_usage'] += sum(task_usage_moment_df['cpu_usage'])
        samples_dicts[moment]['mem_usage'] += sum(task_usage_moment_df['mem_usage'])
        samples_dicts[moment]['disk_io_time'] += sum(task_usage_moment_df['disk_io_time'])
        samples_dicts[moment]['disk_space'] += sum(task_usage_moment_df['mean_local_disk_space'])

samples_df = DataFrame(samples_dicts.values())
print samples_df.info()

samples_df.to_csv(path_or_buf='results/random_sampling_machineid_'+str(machine_id)+'.csv',index=False)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 200 entries, 0 to 199
Data columns (total 9 columns):
cpu_requested               200 non-null float64
cpu_usage                   200 non-null float64
disk                        200 non-null float64
disk_io_time                200 non-null float64
disk_space                  200 non-null float64
mem_requested               200 non-null float64
mem_usage                   200 non-null float64
number_of_task_requested    200 non-null int64
time                        200 non-null int64
dtypes: float64(7), int64(2)None


In [38]:
fig = plt.figure()
ax = fig.add_subplot(111)
# ax.plot(samples_df['time'], samples_df['cpu_requested'], label='cpu requested')
# ax.plot(samples_df['time'], samples_df['cpu_available'], label='cpu available')
ax.plot(samples_df['time'], samples_df['cpu_usage'], label='cpu usage')
plt.xlim(min(samples_df['time']), max(samples_df['time']))
plt.legend()
plt.show()

In [20]:
df = read_csv('results/random_sampling_machineid_'+str(machine_id)+'.csv',index_col=False)
fig = plt.figure()
ax = fig.add_subplot(111)
ax.plot(df['time'], df['cpu_requested'], label='cpu requested')
ax.plot(df['time'], df['mem_requested'], label='mem requested')
ax.plot(df['time'], df['disk'], label='disk space requested')
plt.xlim(min(df['time']), max(df['time']))
plt.title('machine ID = ' +str(machine_id))
plt.legend()
plt.show()

In [106]:
(task_event_df['machine_id'])

0     4155527081
1      329150663
2     3938719206
3      351618647
4      431052910
5      257348783
6     5655258253
7     3550322224
8        1303745
9     3894543095
10     336025676
11    3405236527
12     431081448
13      84899647
14       1268205
...
450131           NaN
450132    4802517408
450133     258727769
450134           NaN
450135     258727769
450136       2850406
450137           NaN
450138    4820204869
450139    4820223869
450140     273549095
450141    1436472882
450142           NaN
450143    1436472882
450144           NaN
450145           NaN
Name: machine_id, Length: 450146, dtype: float64

In [6]:
len(sample_moments)

200

In [41]:
task_events_df['disk']

0          NaN
1          NaN
2     0.000424
3     0.000424
4          NaN
5          NaN
6          NaN
7          NaN
8          NaN
9          NaN
10         NaN
11         NaN
12         NaN
13         NaN
14         NaN
...
450131    0.000115
450132    0.000011
450133    0.000077
450134    0.000077
450135    0.000011
450136    0.000038
450137    0.000038
450138    0.000386
450139    0.000386
450140    0.000386
450141    0.000019
450142    0.000019
450143    0.000011
450144    0.000215
450145    0.000215
Name: disk, Length: 450146, dtype: float64

In [23]:
fp

'task_events/part-00000-of-00500.csv.gz'

In [12]:
#lay thong tin CPU, RAM available cua machine
from pandas import read_csv
from os import path
import sys
from os import listdir, chdir
from collections import OrderedDict
from pandas import DataFrame
from random import randint, sample, seed

data_directory = 'machine_events'
results_directory = 'results'

interval = 1

maxtime_stamp = 2506199602822

seed(83)
sample_moments = range(0,maxtime_stamp,1000000* (int)(interval)) # lay mau theo giay
# sample_moments = sorted(sample(xrange(maxtime_stamp), 200))

machine_id = 1268205
machines_dict = {}
samples_dicts = OrderedDict([])
sample_moments_iterator = iter(sample_moments)
current_sample_moment = next(sample_moments_iterator)

machine_events_csv_colnames=['time', 'machine_id', 'event_type', 'platform_id', 'cpu', 'mem']

# %%time
for fn in sorted(listdir(data_directory)):
    fp = path.join(data_directory, fn)
    machine_events_df = read_csv(fp, header=None, index_col=False, compression='gzip', names=machine_events_csv_colnames)
    print 'reading file '+fp
    for index, event in machine_events_df.iterrows():

        if current_sample_moment is not None and event['time'] > current_sample_moment:
            tmp_machines_df = DataFrame(machines_dict.values())
            if not tmp_machines_df.empty:
                samples_dicts[current_sample_moment] = ({'time' : current_sample_moment,
                                                         'cpu_available' : (tmp_machines_df['cpu']).sum(),
                                                         'mem_available' : (tmp_machines_df['mem']).sum()})
            else:
                samples_dicts[current_sample_moment] = ({'cpu_available' : 0,
                                                         'mem_available' : 0})
            try:
                current_sample_moment = next(sample_moments_iterator)
            except StopIteration:
                current_sample_moment = None
        if event['machine_id'] == machine_id:
            if event['event_type'] in [0, 2]:
                machines_dict[event['machine_id']] = {'machine_id' : event['machine_id'],
                                                      'cpu' : event['cpu'], 'mem' : event['mem']}
            elif event['event_type'] in [1]:
                print 'co bi tat'
                del machines_dict[event['machine_id']]


    if current_sample_moment is None:
        break

samples_df = DataFrame(samples_dicts.values())
print samples_df.info()

reading file machine_events/part-00000-of-00001.csv.gz
<class 'pandas.core.frame.DataFrame'>
Int64Index: 25303 entries, 0 to 25302
Data columns (total 3 columns):
cpu_available    25303 non-null float64
mem_available    25303 non-null float64
time             25303 non-null int64
dtypes: float64(2), int64(1)

None


In [16]:
event['time']

2506135493517

In [17]:
current_sample_moment

25303000000

In [18]:
machines_dict

{1268205: {'cpu': 0.5, 'machine_id': 1268205, 'mem': 0.4995}}

In [20]:
next(sample_moments_iterator)

25305000000

In [24]:
max(sample_moments)

2506199000000

In [ ]:
samples_df